In [1]:
from __future__ import print_function

import os
import sys
import argparse
import time
import math

import tensorboard_logger as tb_logger
import torch
import torch.backends.cudnn as cudnn
from torchvision import transforms, datasets

from util import TwoCropTransform, AverageMeter
from util import adjust_learning_rate, warmup_learning_rate
from util import set_optimizer, save_model
from resnet_model import SupConResNet
from contrastive_loss import SupConLoss

In [2]:
def parse_option():
    parser = argparse.ArgumentParser('argument for training')

    parser.add_argument('--print_freq', type=int, default=10, help='print frequency')
    parser.add_argument('--save_freq', type=int, default=25, help='save frequency')
    parser.add_argument('--batch_size', type=int, default=2, help='batch_size')
    parser.add_argument('--num_workers', type=int, default=1, help='num of workers to use')
    parser.add_argument('--epochs', type=int, default=50, help='number of training epochs')

    # optimization
    parser.add_argument('--learning_rate', type=float, default=0.05, help='learning rate')
    parser.add_argument('--lr_decay_epochs', type=str, default='700,800,900', help='where to decay lr, can be a list')
    parser.add_argument('--lr_decay_rate', type=float, default=0.1, help='decay rate for learning rate')
    parser.add_argument('--weight_decay', type=float, default=1e-4, help='weight decay')
    parser.add_argument('--momentum', type=float, default=0.9, help='momentum')

    # model dataset
    parser.add_argument('--model', type=str, default='resnet18')
    parser.add_argument('--dataset', type=str, default='path', help='dataset')
    parser.add_argument('--mean', default=(0, 0, 0), type=str, help='mean of dataset in path in form of str tuple')
    parser.add_argument('--std', default=(1, 1, 1), type=str, help='std of dataset in path in form of str tuple')
    parser.add_argument('--data_folder', type=str, default='/home/ivoks/Desktop/cowsNose', help='path to custom dataset')
    parser.add_argument('--size', type=int, default=256, help='parameter for RandomResizedCrop')
    parser.add_argument('--method', type=str, default='SupCon', help='choose method')
    parser.add_argument('--temp', type=float, default=0.07, help='temperature for loss function')
    parser.add_argument('--cosine', action='store_true', help='using cosine annealing. Learning Rate Scheduler!')
    parser.add_argument('--warm', action='store_true', help='warm-up for large batch training')
    parser.add_argument('--trial', type=str, default='0', help='id for recording multiple runs')

    opt, unknown = parser.parse_known_args()

    # set the path according to the environment
    opt.model_path = './save/SupCon/{}_models'.format(opt.dataset)
    opt.tb_path = './save/SupCon/{}_tensorboard'.format(opt.dataset)

    iterations = opt.lr_decay_epochs.split(',')
    opt.lr_decay_epochs = list([])
    for it in iterations:
        opt.lr_decay_epochs.append(int(it))

    opt.model_name = '{}_{}_lr_{}_decay_{}_bsz_{}_temp_{}_trial_{}'. \
        format(opt.method, opt.model, opt.learning_rate,
               opt.weight_decay, opt.batch_size, opt.temp, opt.trial)

    if opt.cosine:
        opt.model_name = '{}_cosine'.format(opt.model_name)

    # warm-up for large-batch training,
#     if opt.batch_size > 256:
#         opt.warm = True
#     if opt.warm:
#         opt.model_name = '{}_warm'.format(opt.model_name)
#         opt.warmup_from = 0.01
#         opt.warm_epochs = 10
#         if opt.cosine:
#             eta_min = opt.learning_rate * (opt.lr_decay_rate ** 3)
#             opt.warmup_to = eta_min + (opt.learning_rate - eta_min) * (
#                     1 + math.cos(math.pi * opt.warm_epochs / opt.epochs)) / 2
#         else:
#             opt.warmup_to = opt.learning_rate

    opt.tb_folder = os.path.join(opt.tb_path, opt.model_name)
    if not os.path.isdir(opt.tb_folder):
        os.makedirs(opt.tb_folder)

    opt.save_folder = os.path.join(opt.model_path, opt.model_name)
    if not os.path.isdir(opt.save_folder):
        os.makedirs(opt.save_folder)

    return opt

In [3]:
def set_loader(opt):
    mean = opt.mean
    std = opt.std
    normalize = transforms.Normalize(mean=mean, std=std)

    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(size=opt.size, scale=(0.8, 1.)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
            transforms.RandomRotation(10)
        ], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        normalize,
    ])
#     train_transform = transforms.Compose([
#         transforms.ToTensor(),
#         normalize,
#     ])

    train_dataset = datasets.ImageFolder(root=opt.data_folder, transform=TwoCropTransform(train_transform))
#     train_dataset = datasets.ImageFolder(root=opt.data_folder, transform=train_transform)
    train_sampler = None
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=opt.batch_size, shuffle=(train_sampler is None),
        num_workers=opt.num_workers, pin_memory=True, sampler=train_sampler)

    return train_loader

In [4]:
def set_model(opt):
    model = SupConResNet(name=opt.model)
    criterion = SupConLoss(temperature=opt.temp)
    clss_criterion = torch.nn.CrossEntropyLoss()
    # enable synchronized Batch Normalization

    if torch.cuda.is_available():
        model = model.cuda()
        criterion = criterion.cuda()
        clss_criterion = clss_criterion.cuda()
        cudnn.benchmark = True

    return model, criterion, clss_criterion

In [5]:
def train(train_loader, model, criterion, clss_criterion, optimizer, epoch, opt):
    """one epoch training"""
    model.train()

    batch_time = AverageMeter()
    losses = AverageMeter()
    
    class_losses = AverageMeter()

    end = time.time()
    for idx, (images, labels) in enumerate(train_loader):
#         print(labels)
        images = torch.cat([images[0], images[1]], dim=0)
        if torch.cuda.is_available():
            images = images.cuda(non_blocking=True)
            labels = labels.cuda(non_blocking=True)
        bsz = labels.shape[0]
        warmup_learning_rate(opt, epoch, idx, len(train_loader), optimizer)
        features, clss = model(images)
        f1, f2 = torch.split(features, [bsz, bsz], dim=0)
        features = torch.cat([f1.unsqueeze(1), f2.unsqueeze(1)], dim=1)
        loss = criterion(features, labels)

        
        # classification loss
#         print('images.shape:{}'.format(images.shape))
#         print('labels.shape:{}'.format(labels.shape))
#         print('features.shape:{}'.format(features.shape))
#         print('clss.shape:{}'.format(clss.shape))

        labels = torch.cat([labels, labels], dim=0)
        class_loss = clss_criterion(clss, labels)
        
        # update metric
        losses.update(loss.item(), bsz)
        
        class_losses.update(class_loss.item(), bsz)
        
        # SGD
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        class_loss.backward(retain_graph=True)
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # print info
        if (idx + 1) % opt.print_freq == 0:
#             print('Train: [{0}][{1}/{2}]\t'
#                   'Batch-Time: {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
#                   'loss {loss.val:.3f} ({loss.avg:.3f})'.format(epoch, idx + 1, len(train_loader),
#                                                                 batch_time=batch_time, loss=losses))
            print('Train: [{0}][{1}/{2}]\t'
                  'Batch-Time: {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Total loss {loss:.3f} ({loss:.3f})\t'
                  'contractive loss {contraloss.val:.3f} ({contraloss.avg:.3f})\t'
                  'classification loss {class_loss.val:.3f} ({class_loss.avg:.3f})\t'.format(epoch, idx + 1, len(train_loader),
                                                                batch_time=batch_time, loss=losses.val+class_losses.val, 
                                                                 contraloss=losses, class_loss=class_losses))
            sys.stdout.flush()

    return losses.avg, class_losses.avg

In [6]:
opt = parse_option()

# build data loader
train_loader = set_loader(opt)

# build model and criterion
model, criterion, clss_criterion= set_model(opt)

# build optimizer
optimizer = set_optimizer(opt, model)

# tensorboard
logger = tb_logger.Logger(logdir=opt.tb_folder, flush_secs=2)

# training routine
for epoch in range(1, opt.epochs + 1):
    adjust_learning_rate(opt, optimizer, epoch)

    # train for one epoch
    time1 = time.time()
    loss, class_loss = train(train_loader, model, criterion, clss_criterion, optimizer, epoch, opt)
    time2 = time.time()
    print('epoch {}, total time {:.2f}'.format(epoch, time2 - time1))

    # tensorboard logger
    logger.log_value('contractive loss', loss, epoch)
    logger.log_value('classification loss', class_loss, epoch)
    logger.log_value('learning_rate', optimizer.param_groups[0]['lr'], epoch)

#         if epoch % opt.save_freq == 0:
#             save_file = os.path.join(
#                 opt.save_folder, 'ckpt_epoch_{epoch}.pth'.format(epoch=epoch))
#             save_model(model, optimizer, opt, epoch, save_file)

# save the last model
save_file = os.path.join(opt.save_folder, 'last.pth')
save_model(model, optimizer, opt, opt.epochs, save_file)

Train: [1][10/145]	Batch-Time: 0.307 (0.810)	Total loss 5.703 (5.703)	contractive loss 1.099 (1.153)	classification loss 4.605 (4.605)	
Train: [1][20/145]	Batch-Time: 0.307 (0.562)	Total loss 5.704 (5.704)	contractive loss 1.099 (1.126)	classification loss 4.606 (4.605)	
Train: [1][30/145]	Batch-Time: 0.316 (0.481)	Total loss 5.704 (5.704)	contractive loss 1.099 (1.117)	classification loss 4.605 (4.605)	
Train: [1][40/145]	Batch-Time: 0.310 (0.441)	Total loss 5.703 (5.703)	contractive loss 1.099 (1.112)	classification loss 4.605 (4.605)	
Train: [1][50/145]	Batch-Time: 0.309 (0.415)	Total loss 5.704 (5.704)	contractive loss 1.099 (1.109)	classification loss 4.606 (4.605)	
Train: [1][60/145]	Batch-Time: 0.310 (0.397)	Total loss 5.704 (5.704)	contractive loss 1.099 (1.108)	classification loss 4.605 (4.605)	
Train: [1][70/145]	Batch-Time: 0.311 (0.386)	Total loss 5.703 (5.703)	contractive loss 1.099 (1.106)	classification loss 4.605 (4.605)	
Train: [1][80/145]	Batch-Time: 0.310 (0.378)	Tot

Train: [5][50/145]	Batch-Time: 0.339 (0.330)	Total loss 5.704 (5.704)	contractive loss 1.098 (1.099)	classification loss 4.605 (4.605)	
Train: [5][60/145]	Batch-Time: 0.333 (0.330)	Total loss 5.704 (5.704)	contractive loss 1.099 (1.099)	classification loss 4.606 (4.605)	
Train: [5][70/145]	Batch-Time: 0.317 (0.330)	Total loss 5.704 (5.704)	contractive loss 1.099 (1.099)	classification loss 4.606 (4.605)	
Train: [5][80/145]	Batch-Time: 0.317 (0.328)	Total loss 5.704 (5.704)	contractive loss 1.099 (1.099)	classification loss 4.605 (4.605)	
Train: [5][90/145]	Batch-Time: 0.317 (0.327)	Total loss 5.703 (5.703)	contractive loss 1.098 (1.099)	classification loss 4.605 (4.605)	
Train: [5][100/145]	Batch-Time: 0.316 (0.326)	Total loss 5.703 (5.703)	contractive loss 1.098 (1.099)	classification loss 4.605 (4.605)	
Train: [5][110/145]	Batch-Time: 0.316 (0.326)	Total loss 5.703 (5.703)	contractive loss 1.099 (1.099)	classification loss 4.604 (4.605)	
Train: [5][120/145]	Batch-Time: 0.316 (0.325)	

Train: [9][90/145]	Batch-Time: 0.318 (0.317)	Total loss 5.399 (5.399)	contractive loss 0.794 (0.964)	classification loss 4.606 (4.605)	
Train: [9][100/145]	Batch-Time: 0.318 (0.317)	Total loss 5.949 (5.949)	contractive loss 1.344 (0.970)	classification loss 4.605 (4.605)	
Train: [9][110/145]	Batch-Time: 0.318 (0.317)	Total loss 5.728 (5.728)	contractive loss 1.122 (0.941)	classification loss 4.605 (4.605)	
Train: [9][120/145]	Batch-Time: 0.318 (0.317)	Total loss 5.338 (5.338)	contractive loss 0.733 (0.910)	classification loss 4.605 (4.605)	
Train: [9][130/145]	Batch-Time: 0.318 (0.317)	Total loss 5.137 (5.137)	contractive loss 0.533 (0.932)	classification loss 4.605 (4.605)	
Train: [9][140/145]	Batch-Time: 0.319 (0.317)	Total loss 5.075 (5.075)	contractive loss 0.470 (0.926)	classification loss 4.605 (4.605)	
epoch 9, total time 46.03
Train: [10][10/145]	Batch-Time: 0.318 (0.309)	Total loss 6.124 (6.124)	contractive loss 1.520 (0.694)	classification loss 4.605 (4.605)	
Train: [10][20/1

Train: [13][120/145]	Batch-Time: 0.318 (0.317)	Total loss 4.707 (4.707)	contractive loss 0.101 (0.803)	classification loss 4.605 (4.605)	
Train: [13][130/145]	Batch-Time: 0.318 (0.317)	Total loss 5.867 (5.867)	contractive loss 1.263 (0.810)	classification loss 4.604 (4.605)	
Train: [13][140/145]	Batch-Time: 0.318 (0.317)	Total loss 4.749 (4.749)	contractive loss 0.145 (0.814)	classification loss 4.604 (4.605)	
epoch 13, total time 46.01
Train: [14][10/145]	Batch-Time: 0.318 (0.306)	Total loss 5.465 (5.465)	contractive loss 0.861 (0.914)	classification loss 4.604 (4.605)	
Train: [14][20/145]	Batch-Time: 0.318 (0.312)	Total loss 4.791 (4.791)	contractive loss 0.186 (0.862)	classification loss 4.605 (4.605)	
Train: [14][30/145]	Batch-Time: 0.318 (0.314)	Total loss 6.075 (6.075)	contractive loss 1.470 (0.780)	classification loss 4.606 (4.605)	
Train: [14][40/145]	Batch-Time: 0.318 (0.315)	Total loss 5.310 (5.310)	contractive loss 0.705 (0.733)	classification loss 4.605 (4.605)	
Train: [14]

epoch 17, total time 46.06
Train: [18][10/145]	Batch-Time: 0.318 (0.304)	Total loss 4.764 (4.764)	contractive loss 0.160 (1.107)	classification loss 4.605 (4.605)	
Train: [18][20/145]	Batch-Time: 0.319 (0.310)	Total loss 5.710 (5.710)	contractive loss 1.105 (1.055)	classification loss 4.605 (4.605)	
Train: [18][30/145]	Batch-Time: 0.317 (0.313)	Total loss 5.748 (5.748)	contractive loss 1.143 (1.006)	classification loss 4.605 (4.605)	
Train: [18][40/145]	Batch-Time: 0.317 (0.314)	Total loss 5.445 (5.445)	contractive loss 0.841 (0.951)	classification loss 4.605 (4.605)	
Train: [18][50/145]	Batch-Time: 0.316 (0.314)	Total loss 5.115 (5.115)	contractive loss 0.509 (0.887)	classification loss 4.606 (4.605)	
Train: [18][60/145]	Batch-Time: 0.317 (0.315)	Total loss 5.740 (5.740)	contractive loss 1.136 (0.961)	classification loss 4.605 (4.605)	
Train: [18][70/145]	Batch-Time: 0.316 (0.315)	Total loss 5.673 (5.673)	contractive loss 1.069 (0.982)	classification loss 4.604 (4.605)	
Train: [18][80

...

Train: [30][100/145]	Batch-Time: 0.318 (0.317)	Total loss 4.651 (4.651)	contractive loss 0.046 (0.709)	classification loss 4.605 (4.604)	
Train: [30][110/145]	Batch-Time: 0.318 (0.317)	Total loss 5.846 (5.846)	contractive loss 1.241 (0.718)	classification loss 4.605 (4.605)	
Train: [30][120/145]	Batch-Time: 0.318 (0.317)	Total loss 4.818 (4.818)	contractive loss 0.215 (0.698)	classification loss 4.603 (4.604)	
Train: [30][130/145]	Batch-Time: 0.318 (0.317)	Total loss 4.793 (4.793)	contractive loss 0.188 (0.696)	classification loss 4.604 (4.605)	
Train: [30][140/145]	Batch-Time: 0.319 (0.317)	Total loss 4.618 (4.618)	contractive loss 0.014 (0.678)	classification loss 4.604 (4.604)	
epoch 30, total time 46.04
Train: [31][10/145]	Batch-Time: 0.317 (0.305)	Total loss 4.650 (4.650)	contractive loss 0.045 (0.530)	classification loss 4.605 (4.605)	
Train: [31][20/145]	Batch-Time: 0.319 (0.312)	Total loss 4.638 (4.638)	contractive loss 0.033 (0.563)	classification loss 4.605 (4.604)	
Train: [3

Train: [34][130/145]	Batch-Time: 0.317 (0.318)	Total loss 4.770 (4.770)	contractive loss 0.166 (0.721)	classification loss 4.604 (4.604)	
Train: [34][140/145]	Batch-Time: 0.317 (0.318)	Total loss 5.543 (5.543)	contractive loss 0.940 (0.730)	classification loss 4.602 (4.604)	
epoch 34, total time 46.08
Train: [35][10/145]	Batch-Time: 0.316 (0.307)	Total loss 6.305 (6.305)	contractive loss 1.701 (0.502)	classification loss 4.604 (4.604)	
Train: [35][20/145]	Batch-Time: 0.316 (0.313)	Total loss 4.688 (4.688)	contractive loss 0.083 (0.601)	classification loss 4.605 (4.605)	
Train: [35][30/145]	Batch-Time: 0.316 (0.314)	Total loss 4.724 (4.724)	contractive loss 0.120 (0.602)	classification loss 4.604 (4.604)	
Train: [35][40/145]	Batch-Time: 0.317 (0.315)	Total loss 4.678 (4.678)	contractive loss 0.076 (0.666)	classification loss 4.603 (4.604)	
Train: [35][50/145]	Batch-Time: 0.317 (0.315)	Total loss 5.822 (5.822)	contractive loss 1.218 (0.666)	classification loss 4.605 (4.604)	
Train: [35][

Train: [39][20/145]	Batch-Time: 0.317 (0.312)	Total loss 5.987 (5.987)	contractive loss 1.382 (1.187)	classification loss 4.604 (4.605)	
Train: [39][30/145]	Batch-Time: 0.316 (0.314)	Total loss 5.666 (5.666)	contractive loss 1.062 (1.194)	classification loss 4.604 (4.604)	
Train: [39][40/145]	Batch-Time: 0.316 (0.314)	Total loss 5.611 (5.611)	contractive loss 1.005 (1.138)	classification loss 4.606 (4.605)	
Train: [39][50/145]	Batch-Time: 0.316 (0.315)	Total loss 5.743 (5.743)	contractive loss 1.137 (1.121)	classification loss 4.606 (4.605)	
Train: [39][60/145]	Batch-Time: 0.317 (0.315)	Total loss 5.534 (5.534)	contractive loss 0.928 (1.053)	classification loss 4.606 (4.605)	
Train: [39][70/145]	Batch-Time: 0.316 (0.315)	Total loss 7.051 (7.051)	contractive loss 2.448 (1.011)	classification loss 4.603 (4.605)	
Train: [39][80/145]	Batch-Time: 0.316 (0.316)	Total loss 4.965 (4.965)	contractive loss 0.359 (0.991)	classification loss 4.606 (4.605)	
Train: [39][90/145]	Batch-Time: 0.316 (0.

Train: [43][50/145]	Batch-Time: 0.317 (0.315)	Total loss 5.396 (5.396)	contractive loss 0.792 (0.577)	classification loss 4.604 (4.604)	
Train: [43][60/145]	Batch-Time: 0.317 (0.315)	Total loss 6.122 (6.122)	contractive loss 1.517 (0.616)	classification loss 4.605 (4.604)	
Train: [43][70/145]	Batch-Time: 0.317 (0.315)	Total loss 6.390 (6.390)	contractive loss 1.787 (0.676)	classification loss 4.604 (4.604)	
Train: [43][80/145]	Batch-Time: 0.317 (0.316)	Total loss 4.673 (4.673)	contractive loss 0.067 (0.670)	classification loss 4.606 (4.604)	
Train: [43][90/145]	Batch-Time: 0.317 (0.316)	Total loss 6.630 (6.630)	contractive loss 2.024 (0.662)	classification loss 4.606 (4.604)	
Train: [43][100/145]	Batch-Time: 0.317 (0.316)	Total loss 4.628 (4.628)	contractive loss 0.024 (0.631)	classification loss 4.604 (4.604)	
Train: [43][110/145]	Batch-Time: 0.317 (0.316)	Total loss 5.406 (5.406)	contractive loss 0.801 (0.629)	classification loss 4.605 (4.604)	
Train: [43][120/145]	Batch-Time: 0.316 

Train: [47][80/145]	Batch-Time: 0.316 (0.316)	Total loss 5.584 (5.584)	contractive loss 0.982 (0.630)	classification loss 4.602 (4.604)	
Train: [47][90/145]	Batch-Time: 0.316 (0.316)	Total loss 4.646 (4.646)	contractive loss 0.040 (0.659)	classification loss 4.606 (4.604)	
Train: [47][100/145]	Batch-Time: 0.317 (0.317)	Total loss 4.680 (4.680)	contractive loss 0.076 (0.647)	classification loss 4.604 (4.604)	
Train: [47][110/145]	Batch-Time: 0.317 (0.317)	Total loss 4.656 (4.656)	contractive loss 0.051 (0.658)	classification loss 4.605 (4.604)	
Train: [47][120/145]	Batch-Time: 0.317 (0.317)	Total loss 4.648 (4.648)	contractive loss 0.042 (0.649)	classification loss 4.605 (4.604)	
Train: [47][130/145]	Batch-Time: 0.317 (0.317)	Total loss 4.951 (4.951)	contractive loss 0.347 (0.654)	classification loss 4.604 (4.604)	
Train: [47][140/145]	Batch-Time: 0.318 (0.317)	Total loss 7.205 (7.205)	contractive loss 2.600 (0.649)	classification loss 4.605 (4.604)	
epoch 47, total time 45.94
Train: [4

In [ ]:
# for idx, (images, labels) in enumerate(train_loader):
#     print(len(images))
#     print(labels.shape)
#     print(images[0])
#     print('##############################################')
#     print(images[1])
#     print('##############################################')
#     print(torch.cat([images[0], images[1]], dim=0))
#     break

In [ ]:
model()

# -----------------------------------------------------------------

In [7]:
## from torchvision import transforms
from PIL import Image
from torchvision.transforms import functional as TF
import matplotlib.pyplot as plt
import numpy as np
import torch

from scipy.spatial.distance import cdist

transform = transforms.Compose([
            # transforms.PILToTensor(),
            transforms.Resize(size=(128, 128)),
            transforms.ToTensor(),
        ])

In [11]:
def preprocess_func(img_path):
    img = Image.open(img_path)
    tensor = transform(img).unsqueeze(0)
    return tensor.to('cpu') #(self.device)

In [12]:
def get_features(images):
    
    if type(images) is not torch.Tensor:
        
        images = np.concatenate(images)
        images = torch.Tensor(images).to('cuda')
    with torch.no_grad():
        res, _ = model(images).detach().cpu().numpy()    
    return res

In [13]:
default='/home/ivoks/Desktop/cowsNose'

def get_all_features(img_dir):
    features = []
    paths = []
    bs = []
    bs_path = []
    e = 0
    for root, dirs, files in os.walk(img_dir):
        for file in files:
            if not file.endswith('jpg') and not file.endswith('png') and not file.endswith('jpeg'):
                print(file + " is passed!")
                continue
            img_path = os.path.join(root, file)
            #if Image.open(img_path).mode != 'RGB':
            #       continue
            img = preprocess_func(img_path)
            if len(bs) < 8:
                bs.append(img)
                bs_path.append(img_path)
            else:
                bs_features = get_features(bs)
                features.extend(bs_features)
                paths.extend(bs_path)
                bs = [img]
                bs_path = [img_path]
                
            e += 1
    bs_features = get_features(bs)
    features.extend(bs_features)
    paths.extend(bs_path)
    print('extracting ' + str(e) + ' features is done!')
    return paths, features

paths, features = get_all_features(default)

AttributeError: 'tuple' object has no attribute 'detach'

In [ ]:
def get_feature(img_path):
    img = preprocess_func(img_path)
    with torch.no_grad():
        res = model(img)[0].detach().cpu().numpy()
    return res

In [ ]:
def get_most_similar(img_path, n=10, distance='euclidean'):
    feature = get_feature(img_path)
    p = cdist(np.array(features), np.expand_dims(feature, axis=0), metric=distance)[:, 0]
    # p = np.sqrt(np.sum((np.array(self.features - feature)) ** 2, axis=1))
    group = zip(p, paths.copy())
    res = sorted(group, key=lambda x: x[0])
    r = res[:n]
    return r

get_most_similar('/home/ivoks/Desktop/XK16-1.jpg')

In [ ]:
paths

In [ ]:
path_1 = '/home/ivoks/Desktop/cowsNose/9/9-0.jpeg'

img = Image.open(path_1)
images = torch.from_numpy(np.asarray(img).transpose([2,0,1])[None,:,:,:]).to('cuda')

In [ ]:
images 

In [ ]:
model

In [ ]:
model(images.to(torch.float32))

In [ ]:
# def get_feature(path):
    
#     img = Image.open(path)
#     tensor = transform(img).unsqueeze(0).to('cuda')
#     with torch.no_grad():
#         res = model(tensor)[0].detach().cpu().numpy()
        
#     return res

In [ ]:
# path_1 = "/home/ivoks/Desktop/cowsNose/9/9-0.jpeg"
# path_2 = "/home/ivoks/Desktop/cowsNose/9/9-2.jpeg"

# feature1 = get_feature(path_1)
# feature2 = get_feature(path_2)


In [ ]:
# np.array([feature1])

In [ ]:
# distance='euclidean'

# p = cdist(np.array([feature1]),np.expand_dims(feature2, axis=0),metric=distance)[:, 0]

# p